In [ ]:
import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os

# 🔍 Fonction d'analyse Fichier Periodic
def analyze_behavior(file_path):
    df = pd.read_csv(file_path)
    
    # Afficher les événements uniques pour vérification
    print("Types d'événements dans les données :")
    print(df['Event'].unique())
    
    # Assurer format cohérent
    df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
    df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time
    
    # Trier les données par date et heure
    df = df.sort_values(['Date', 'Hour']).reset_index(drop=True)
    
    # Créer un identifiant de bloc pour les périodes consécutives
    # Cela va identifier chaque changement de période (Day->Night ou Night->Day)
    df['Period_Block'] = (df['Period'] != df['Period'].shift(1)).cumsum()
    
    # Définir les catégories d'événements
    specific_events = [
        'LeftDuringDispense',
        'RightDuringDispense',
        'LeftWithPellet',
        'RightWithPellet',
        'RightinTimeOut',
        'LeftinTimeOut'
    ]
    
    results = []
    
    # Regrouper par bloc de période (périodes consécutives)
    for (period_block), group in df.groupby('Period_Block'):
        # Le period_type est le même pour tout le groupe (Day ou Night)
        period_type = group['Period'].iloc[0]
        
        left_pokes = group[group['Event'] == 'Left'].shape[0]
        right_pokes = group[group['Event'] == 'Right'].shape[0]
        total_pokes = left_pokes + right_pokes
        correct = (group['Event'] == group['Active_Poke']).sum()
        
        # Comptage des pellets
        pellets = group[group['Event'] == 'Pellet'].shape[0]
        
        # First poke après Pellet
        pellet_indices = group[group['Event'] == 'Pellet'].index
        first_pokes = []
        for idx in pellet_indices:
            if idx + 1 in group.index:
                first_pokes.append(group.loc[idx + 1])
        first_pokes_df = pd.DataFrame(first_pokes)
        first_poke_total = len(first_pokes)
        first_poke_correct = (first_pokes_df['Event'] == first_pokes_df['Active_Poke']).sum() if not first_pokes_df.empty else 0
        
        # Calcul des pourcentages
        pct_first_poke_correct = round((first_poke_correct / first_poke_total) * 100, 2) if first_poke_total > 0 else None
        pct_correct = round((correct / total_pokes) * 100, 2) if total_pokes > 0 else None
        
        # Specific events
        spec_events = group[group['Event'].isin(specific_events)].shape[0]
        
        # Dates de début et fin du bloc
        start_date = group['Date'].min()
        end_date = group['Date'].max()
        
        results.append({
            'Period Block': period_block,
            'Period Type': period_type,
            'Start Date': start_date,
            'End Date': end_date,
            'Pellet': pellets,
            'Pokes': total_pokes,
            'Left Pokes': left_pokes,
            'Right Pokes': right_pokes,
            'Correct': correct,
            '% Correct': pct_correct,
            'First Poke': first_poke_total,
            'First Poke Correct': first_poke_correct,
            '% First Poke Correct': pct_first_poke_correct,
            'Specific Events': spec_events,
        })
    
    # Résumé
    summary_df = pd.DataFrame(results)
    
    # Sauvegarde CSV
    folder = os.path.dirname(file_path)
    base = os.path.splitext(os.path.basename(file_path))[0]
    output_summary = os.path.join(folder, f"{base}_global.csv")
    summary_df.to_csv(output_summary, index=False)
    
    print(f"\n✅ Fichier résumé sauvegardé avec succès !")
    print(f"📍 Chemin : {output_summary}")
    display(summary_df)

# 🎯 Sélection du fichier avec FileChooser
def choose_file_and_analyze():
    def on_file_selected(chooser):
        selected_file = chooser.selected
        print(f"\n📂 Fichier sélectionné : {selected_file}")
        analyze_behavior(selected_file)
    
    fc = FileChooser("C:/Users/AudreyHay/Documents/Carla/FED/", select_default=True,show_only_dirs=False, title="<b>Sélectionnez le fichier CSV</b>")
    display(fc)
    fc.register_callback(on_file_selected)

# 🔽 Lance l'interface
choose_file_and_analyze()

In [ ]:
import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Configuration du style des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def create_sequential_plots(summary_df, output_folder, base_name):
    """Crée les graphiques avec les périodes en séquence day-night"""
    
    # Créer une séquence ordonnée alternant Day/Night
    day_blocks = summary_df[summary_df['Period Type'] == 'Day'].copy()
    night_blocks = summary_df[summary_df['Period Type'] == 'Night'].copy()
    
    # Créer la séquence alternée
    sequential_data = []
    max_periods = max(len(day_blocks), len(night_blocks))
    
    for i in range(max_periods):
        # Ajouter Day i+1 s'il existe
        if i < len(day_blocks):
            day_row = day_blocks.iloc[i].copy()
            day_row['Sequential_Label'] = f'Day {i+1}'
            day_row['Sequential_Order'] = i*2
            sequential_data.append(day_row)
        
        # Ajouter Night i+1 s'il existe
        if i < len(night_blocks):
            night_row = night_blocks.iloc[i].copy()
            night_row['Sequential_Label'] = f'Night {i+1}'
            night_row['Sequential_Order'] = i*2 + 1
            sequential_data.append(night_row)
    
    # Créer le DataFrame séquentiel
    seq_df = pd.DataFrame(sequential_data)
    seq_df = seq_df.sort_values('Sequential_Order').reset_index(drop=True)
    
    # Définir les couleurs
    colors = ['#FFD700' if 'Day' in label else '#4169E1' 
              for label in seq_df['Sequential_Label']]
    
    # Créer le graphique principal avec subplots
    fig = plt.figure(figsize=(20, 16))
    
    # 1. Pourcentage de réponses correctes
    plt.subplot(3, 2, 1)
    bars1 = plt.bar(range(len(seq_df)), seq_df['% Correct'], 
                    color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.title('Pourcentage de Réponses Correctes', fontsize=14, fontweight='bold')
    plt.ylabel('Pourcentage (%)', fontsize=12)
    plt.xticks(range(len(seq_df)), seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 100)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars1, seq_df['% Correct']):
        if pd.notna(value):
            plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # 2. Nombre de pokes gauche et droite
    plt.subplot(3, 2, 2)
    x = np.arange(len(seq_df))
    width = 0.35
    
    plt.bar(x - width/2, seq_df['Left Pokes'], width, 
            label='Pokes Gauche', alpha=0.7, color='green', edgecolor='black', linewidth=0.5)
    plt.bar(x + width/2, seq_df['Right Pokes'], width, 
            label='Pokes Droite', alpha=0.7, color='red', edgecolor='black', linewidth=0.5)
    
    plt.title('Nombre de Pokes Gauche vs Droite', fontsize=14, fontweight='bold')
    plt.ylabel('Nombre de Pokes', fontsize=12)
    plt.xticks(x, seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. Pourcentage de premier poke correct
    plt.subplot(3, 2, 3)
    bars3 = plt.bar(range(len(seq_df)), seq_df['% First Poke Correct'], 
                    color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.title('Pourcentage de Premier Poke Correct', fontsize=14, fontweight='bold')
    plt.ylabel('Pourcentage (%)', fontsize=12)
    plt.xticks(range(len(seq_df)), seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 100)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars3, seq_df['% First Poke Correct']):
        if pd.notna(value):
            plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # 4. Nombre de pokes non requis (Specific Events)
    plt.subplot(3, 2, 4)
    bars4 = plt.bar(range(len(seq_df)), seq_df['Specific Events'], 
                    color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.title('Nombre de Pokes Non Requis (Événements Spécifiques)', fontsize=14, fontweight='bold')
    plt.ylabel('Nombre', fontsize=12)
    plt.xticks(range(len(seq_df)), seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars4, seq_df['Specific Events']):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                f'{int(value)}', ha='center', va='bottom', fontsize=10)
    
    # 5. Total des pokes par période
    plt.subplot(3, 2, 5)
    bars5 = plt.bar(range(len(seq_df)), seq_df['Pokes'], 
                    color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.title('Nombre Total de Pokes par Période', fontsize=14, fontweight='bold')
    plt.ylabel('Nombre de Pokes', fontsize=12)
    plt.xticks(range(len(seq_df)), seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars5, seq_df['Pokes']):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                f'{int(value)}', ha='center', va='bottom', fontsize=10)
    
    # 6. Nombre de pellets obtenus
    plt.subplot(3, 2, 6)
    bars6 = plt.bar(range(len(seq_df)), seq_df['Pellet'], 
                    color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.title('Nombre de Pellets Obtenus', fontsize=14, fontweight='bold')
    plt.ylabel('Nombre de Pellets', fontsize=12)
    plt.xticks(range(len(seq_df)), seq_df['Sequential_Label'], rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars6, seq_df['Pellet']):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                f'{int(value)}', ha='center', va='bottom', fontsize=10)
    
    # Titre général et légende
    fig.suptitle('Analyse Comportementale - Séquence Day/Night', fontsize=18, fontweight='bold', y=0.98)
    
    # Créer une légende générale
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#FFD700', alpha=0.7, edgecolor='black', label='Période Jour'),
        Patch(facecolor='#4169E1', alpha=0.7, edgecolor='black', label='Période Nuit')
    ]
    fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.99, 0.95), fontsize=12)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    
    # Sauvegarder le graphique
    plot_path = os.path.join(output_folder, f"{base_name}_sequential_analysis.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Graphique sauvegardé : {plot_path}")
    
    plt.show()
    
    return seq_df

def create_trend_lines(seq_df, output_folder, base_name):
    """Crée des graphiques en ligne pour montrer les tendances"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    x = range(len(seq_df))
    
    # Séparer les données Day et Night pour les couleurs
    day_mask = seq_df['Period Type'] == 'Day'
    night_mask = seq_df['Period Type'] == 'Night'
    
    # 1. Tendance du pourcentage correct
    axes[0, 0].plot(x, seq_df['% Correct'], 'o-', linewidth=2, markersize=8, alpha=0.8)
    for i, (is_day, value) in enumerate(zip(day_mask, seq_df['% Correct'])):
        color = '#FFD700' if is_day else '#4169E1'
        axes[0, 0].scatter(i, value, color=color, s=100, alpha=0.8, edgecolor='black')
    
    axes[0, 0].set_title('Tendance du Pourcentage Correct', fontweight='bold')
    axes[0, 0].set_ylabel('Pourcentage (%)')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(seq_df['Sequential_Label'], rotation=45, ha='right')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_ylim(0, 100)
    
    # 2. Tendance des pokes totaux
    axes[0, 1].plot(x, seq_df['Pokes'], 's-', linewidth=2, markersize=8, alpha=0.8)
    for i, (is_day, value) in enumerate(zip(day_mask, seq_df['Pokes'])):
        color = '#FFD700' if is_day else '#4169E1'
        axes[0, 1].scatter(i, value, color=color, s=100, alpha=0.8, edgecolor='black')
    
    axes[0, 1].set_title('Tendance du Nombre Total de Pokes', fontweight='bold')
    axes[0, 1].set_ylabel('Nombre de Pokes')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(seq_df['Sequential_Label'], rotation=45, ha='right')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Tendance du premier poke correct
    axes[1, 0].plot(x, seq_df['% First Poke Correct'], '^-', linewidth=2, markersize=8, alpha=0.8)
    for i, (is_day, value) in enumerate(zip(day_mask, seq_df['% First Poke Correct'])):
        color = '#FFD700' if is_day else '#4169E1'
        if pd.notna(value):
            axes[1, 0].scatter(i, value, color=color, s=100, alpha=0.8, edgecolor='black')
    
    axes[1, 0].set_title('Tendance du Premier Poke Correct', fontweight='bold')
    axes[1, 0].set_ylabel('Pourcentage (%)')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(seq_df['Sequential_Label'], rotation=45, ha='right')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim(0, 100)
    
    # 4. Tendance des événements spécifiques
    axes[1, 1].plot(x, seq_df['Specific Events'], 'd-', linewidth=2, markersize=8, alpha=0.8)
    for i, (is_day, value) in enumerate(zip(day_mask, seq_df['Specific Events'])):
        color = '#FFD700' if is_day else '#4169E1'
        axes[1, 1].scatter(i, value, color=color, s=100, alpha=0.8, edgecolor='black')
    
    axes[1, 1].set_title('Tendance des Pokes Non Requis', fontweight='bold')
    axes[1, 1].set_ylabel('Nombre')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(seq_df['Sequential_Label'], rotation=45, ha='right')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Légende générale
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#FFD700', alpha=0.8, edgecolor='black', label='Jour'),
        Patch(facecolor='#4169E1', alpha=0.8, edgecolor='black', label='Nuit')
    ]
    fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.99, 0.95))
    
    plt.suptitle('Tendances Temporelles - Analyse Day/Night', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    
    # Sauvegarder
    trend_path = os.path.join(output_folder, f"{base_name}_trends.png")
    plt.savefig(trend_path, dpi=300, bbox_inches='tight')
    print(f"📈 Graphique des tendances sauvegardé : {trend_path}")
    
    plt.show()

# 🔍 Fonction d'analyse Fichier Periodic (modifiée)
def analyze_behavior(file_path):
    df = pd.read_csv(file_path)
    
    # Vérifier la structure du fichier
    print("📋 Colonnes disponibles dans le fichier :")
    print(df.columns.tolist())
    print(f"\n📊 Forme du DataFrame : {df.shape}")
    print(f"\n👀 Aperçu des premières lignes :")
    display(df.head())
    
    # Vérifier si les colonnes nécessaires existent
    required_columns = ['Event', 'Period', 'Active_Poke', 'MM:DD:YYYY hh:mm:ss', 'Date', 'Hour']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"\n❌ ERREUR : Colonnes manquantes : {missing_columns}")
        print("Veuillez vérifier que votre fichier contient toutes les colonnes nécessaires.")
        return
    
    # Afficher les événements uniques pour vérification
    print(f"\n🎯 Types d'événements dans les données ({len(df['Event'].unique())} types) :")
    print(df['Event'].unique())
    
    # Assurer format cohérent
    df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
    df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time
    
    # Trier les données par date et heure
    df = df.sort_values(['Date', 'Hour']).reset_index(drop=True)
    
    # Créer un identifiant de bloc pour les périodes consécutives
    # Cela va identifier chaque changement de période (Day->Night ou Night->Day)
    df['Period_Block'] = (df['Period'] != df['Period'].shift(1)).cumsum()
    
    # Définir les catégories d'événements
    specific_events = [
        'LeftDuringDispense',
        'RightDuringDispense',
        'LeftWithPellet',
        'RightWithPellet',
        'RightinTimeOut',
        'LeftinTimeOut'
    ]
    
    results = []
    
    # Regrouper par bloc de période (périodes consécutives)
    for (period_block), group in df.groupby('Period_Block'):
        # Le period_type est le même pour tout le groupe (Day ou Night)
        period_type = group['Period'].iloc[0]
        
        left_pokes = group[group['Event'] == 'Left'].shape[0]
        right_pokes = group[group['Event'] == 'Right'].shape[0]
        total_pokes = left_pokes + right_pokes
        correct = (group['Event'] == group['Active_Poke']).sum()
        
        # Comptage des pellets
        pellets = group[group['Event'] == 'Pellet'].shape[0]
        
        # First poke après Pellet
        pellet_indices = group[group['Event'] == 'Pellet'].index
        first_pokes = []
        for idx in pellet_indices:
            if idx + 1 in group.index:
                first_pokes.append(group.loc[idx + 1])
        first_pokes_df = pd.DataFrame(first_pokes)
        first_poke_total = len(first_pokes)
        first_poke_correct = (first_pokes_df['Event'] == first_pokes_df['Active_Poke']).sum() if not first_pokes_df.empty else 0
        
        # Calcul des pourcentages
        pct_first_poke_correct = round((first_poke_correct / first_poke_total) * 100, 2) if first_poke_total > 0 else None
        pct_correct = round((correct / total_pokes) * 100, 2) if total_pokes > 0 else None
        
        # Specific events
        spec_events = group[group['Event'].isin(specific_events)].shape[0]
        
        # Dates de début et fin du bloc
        start_date = group['Date'].min()
        end_date = group['Date'].max()
        
        results.append({
            'Period Block': period_block,
            'Period Type': period_type,
            'Start Date': start_date,
            'End Date': end_date,
            'Pellet': pellets,
            'Pokes': total_pokes,
            'Left Pokes': left_pokes,
            'Right Pokes': right_pokes,
            'Correct': correct,
            '% Correct': pct_correct,
            'First Poke': first_poke_total,
            'First Poke Correct': first_poke_correct,
            '% First Poke Correct': pct_first_poke_correct,
            'Specific Events': spec_events,
        })
    
    # Résumé
    summary_df = pd.DataFrame(results)
    
    # Sauvegarde CSV
    folder = os.path.dirname(file_path)
    base = os.path.splitext(os.path.basename(file_path))[0]
    output_summary = os.path.join(folder, f"{base}_global.csv")
    summary_df.to_csv(output_summary, index=False)
    
    print(f"\n✅ Fichier résumé sauvegardé avec succès !")
    print(f"📍 Chemin : {output_summary}")
    display(summary_df)
    
    # NOUVEAUTÉ : Créer les graphiques séquentiels
    print("\n🎨 Création des graphiques...")
    seq_df = create_sequential_plots(summary_df, folder, base)
    create_trend_lines(seq_df, folder, base)
    
    # Sauvegarder aussi le DataFrame séquentiel
    sequential_path = os.path.join(folder, f"{base}_sequential.csv")
    seq_df.to_csv(sequential_path, index=False)
    print(f"📊 Données séquentielles sauvegardées : {sequential_path}")

# 🎯 Sélection du fichier avec FileChooser
def choose_file_and_analyze():
    def on_file_selected(chooser):
        selected_file = chooser.selected
        print(f"\n📂 Fichier sélectionné : {selected_file}")
        analyze_behavior(selected_file)
    
    fc = FileChooser(".", show_only_dirs=False, title="<b>Sélectionnez le fichier .csv modifié</b>")
    display(fc)
    fc.register_callback(on_file_selected)

# 🔽 Lance l'interface
choose_file_and_analyze()